### 16.1 이진 분류기 훈련하기
타깃 벡터의 값이 두 개 뿐인 이진 분류기. 
- 로지스틱 함수는 함수의 출력을 0과 1사이로 제한하는 효과가 있다. 
- 함수: $P(y_i = 1 | X)=\frac{1}{1+e^-(\beta_0+\beta_1*x)}$
- $P(y_i = 1 | X)$가 0.5보다 크면 1로 예측하고 그렇지 않으면 0으로 예측한다.

사이킷런에서 로지스틱 회귀 모델을 훈련시킨다면 새로운 샘플의 클래스를 예측할 수 있다.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data[:100,:]
target = iris.target[:100]

#feature 표준화
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 로지스틱 회귀 모델 생성
logistic_regression = LogisticRegression(random_state=20200920)
model = logistic_regression.fit(features_standardized, target)

#새로운 샘플
new_observation = [[.5, .5, .5, .5]]
print(model.predict(new_observation), model.predict_proba(new_observation))

[1] [[0.17738424 0.82261576]]


new_observation이라는 샘플을 클래스 1로 예측하였고 그러한 근거를 predict_proba를 통해 확인할 수 있다. 0일 확률은 17.7%이고 1일 확률은 82.3%이다.